In [1]:
import pandas as pd 
import numpy as np

In [2]:
data = pd.read_csv("data/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data["review"]

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [21]:
import re
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_word = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marwakhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/marwakhan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
def rm_stop_words(sentence):
    words = sentence.split()
    filter_words = []
    filter_words = [word for word in words if word not in stop_word]
    
    return ' '.join(filter_words)

In [23]:
def rm_punctuation(data):
    return data.translate(str.maketrans('','',string.punctuation))

In [24]:
def remove_html(data):
    patterns = re.compile('<.*?>')
    return patterns.sub(r'',data)

In [25]:
def rm_url(data):
    patterns = re.compile(r"https?://\s+www\.\s+")
    return patterns.sub(r'',data)

steming


In [26]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

porter = PorterStemmer()

def stem_sentence(sentence):
    tokens = word_tokenize(sentence)
    stemmed_token = [porter.stem(token) for token in tokens]
    return ' '.join(stemmed_token)
 

In [27]:
def preprocessed(data):
    data['review'] = data['review'].str.lower()
    data['review'] = data['review'].apply(remove_html)
    data['review'] = data['review'].apply(rm_url)
    data['review'] = data['review'].apply(rm_punctuation)
    data['review'] = data['review'].apply(rm_stop_words)
    data['review'] = data['review'].apply(stem_sentence)
    return data

In [28]:
refine_data = preprocessed(data)

In [29]:
new_data = refine_data.copy()
new_data.to_csv("transformed.csv")

In [30]:
df = pd.read_csv("transformed_data/transformed.csv")
df.head()

,Unnamed: 0,review,sentiment
0,0,one review mention watch 1 oz episod youll hoo...,positive
1,1,wonder littl product film techniqu unassum old...,positive
2,2,thought wonder way spend time hot summer weeke...,positive
3,3,basic there famili littl boy jake think there ...,negative
4,4,petter mattei love time money visual stun film...,positive


In [35]:
y = df['sentiment']
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [36]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [40]:
train = vectorizer.fit_transform(df['review'])


In [41]:
train

<50000x181751 sparse matrix of type '<class 'numpy.int64'>'
	with 4779294 stored elements in Compressed Sparse Row format>

In [42]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(train,y,test_size=0.2,random_state=42)

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
rf = RandomForestClassifier(verbose=1,n_jobs=-1)
rf.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.8min finished


RandomForestClassifier(n_jobs=-1, verbose=1)

In [48]:
from sklearn.metrics import accuracy_score
y_predict = rf.predict(X_test)
accuracy = accuracy_score(y_predict,y_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.0s finished


In [49]:
accuracy

0.8511